In [ ]:
import geopandas as gpd

import pandas as pd

In [ ]:
parq = gpd.read_parquet(
    "/media/muskrat/T7 Shield/eco_data/v3/native/GAP/init/4500.parquet"
)

In [ ]:
parq

In [ ]:
# remove duplicates from parq

parq = parq.drop_duplicates(subset=["intGapOrigin", "geometry"])

parq

In [ ]:
# species dataframe = group by scientific name "Lithobates catesbeianus" parq

# species = parq[parq["scientific_name"] == "Lithobates catesbeianus"]

# species

In [ ]:
unique_species = parq["scientific_name"].unique().tolist()

unique_species

In [ ]:
# remove items in unique_species that contain more than 2 words

unique_species = [x for x in unique_species if len(x.split()) < 3]

unique_species

In [ ]:
# split species into dataframes based on how many rows they have
singles = pd.DataFrame()
multiples = pd.DataFrame()
for species in unique_species:
    species_df = parq[parq["scientific_name"] == species]

    origin_list = species_df["intGapOrigin"].tolist()
    unique_origin = set(origin_list)

    presence_list = species_df["intGapPres"].tolist()
    unique_presence = set(presence_list)

    # # only keep if 1 row and intGapOrigin is 1(native)
    if len(species_df) == 1 and unique_origin == {1} and unique_presence == {1}:
        singles = singles.append(species_df)
    # only keep if more than 1 row and unique_list contains 1
    elif len(species_df) > 1 and 1 in unique_origin and 1 in unique_presence:
        multiples = multiples.append(species_df)

In [ ]:
singles

In [ ]:
multiples

In [ ]:
multiples_species_unique = multiples["scientific_name"].unique().tolist()

# split multiples into dataframes based on if they are all native or native and non-native
multiples_native = pd.DataFrame()
multiples_non_native = pd.DataFrame()

for species in multiples_species_unique:
    species_df = multiples[multiples["scientific_name"] == species]

    list = species_df["intGapOrigin"].tolist()
    # if list contains value other than 1, then append to multiples_non_native
    unique_list = set(list)
    if len(unique_list) > 1:
        multiples_non_native = multiples_non_native.append(species_df)
    else:
        multiples_native = multiples_native.append(species_df)

In [ ]:
multiples_native

In [ ]:
multiples_non_native

In [ ]:
# df = multiples_non_native[
#     multiples_non_native["scientific_name"] == "Lithobates catesbeianus"
# ]

# df[0:1]

In [ ]:
# concat singles and multiples_native

clean = pd.concat([singles, multiples_native])

clean

In [ ]:
# drop rows where intGapOrigin is not 1

clean = clean[clean["intGapOrigin"] == 1]

# drop columns intGapPres, intGapRepro, intGapSeas

clean = clean.drop(
    columns=["intGapPres", "intGapRepro", "intGapSeas", "Reproduction", "Season"]
)

clean

In [ ]:
clean.to_parquet(
    "/media/muskrat/T7 Shield/eco_data/v3/native/GAP/clean/4500_pre_clean.parquet"
)

In [ ]:
multiples_non_native.to_parquet(
    "/media/muskrat/T7 Shield/eco_data/v3/native/GAP/clean/4500_check.parquet"
)

In [ ]:
# filter by intGapOrigin = 1 (native) (and Origin = Native for double check) and intGapPres = 1 (present) (and Presence = Known/extant for double check)

In [ ]:
df.plot()

In [ ]:
df.crs

# check units
# df.crs.axis_info[0].unit_name

In [ ]:
# converts crs to epsg:4326
df = df.to_crs("EPSG:4326")
df.crs

In [ ]:
df.plot()

In [ ]:
ecomap_loc = "/media/muskrat/T7 Shield/eco_data/ecomap_final/eco_map.geojson"

eco_map = gpd.read_file(ecomap_loc)

In [ ]:
eco_map

In [ ]:
eco_map.crs

In [ ]:
# convert df to geodataframe maybe not needed
df = gpd.GeoDataFrame(df, geometry="geometry")

In [ ]:
# plot ecomap and df on same map
base = eco_map.plot(color="white", edgecolor="black")
xmin, ymin, xmax, ymax = (-120, 30, -100, 45)

ax = df.plot(ax=base, color="red", alpha=0.4)

# set the x and y limits of the plot to the specified bounding box coordinates
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
# plot the GeoDataFrame with the specified bounding box
# df.plot(ax=base, color='red', extent=[xmin, xmax, ymin, ymax])

# df.plot(ax=base, color='red')

In [ ]:
df

In [ ]:
# check if df geometry intersects with ecomap geometry

intersects = gpd.sjoin(df, eco_map)

In [ ]:
intersects.head()

In [ ]:
# put unique values of unique_id in intersects into a list

unique_ids = list(intersects["unique_id"].unique())
unique_ids

In [ ]:
# create dataframe from eco_map that only contains the unique ids in unique_ids

eco_map_unique = eco_map[eco_map["unique_id"].isin(unique_ids)]
# eco_map_unique

In [ ]:
eco_map_unique.head()

In [ ]:
overlay = gpd.overlay(df, eco_map, how="intersection")

In [ ]:
overlay.plot(alpha=0.5, edgecolor="k", cmap="tab10")

In [ ]:
overlay

In [ ]:
o_area = overlay.area.sum()

o_area

In [ ]:
# add area column to overlay dataframe

overlay["area"] = overlay.geometry.area

In [ ]:
# find row with max area in overlay dataframe

overlay.loc[overlay["area"].idxmax()]

In [ ]:
# print overlay row

overlay.iloc[6]

In [ ]:
# create a new dataframe from overlay where the first column is unique_id and the second column is the area of all the rows in overlay that have the same unique_id

overlay_areas = overlay[["unique_id", "area"]].groupby("unique_id").sum()

In [ ]:
overlay_areas

In [ ]:
# add an area column to eco_map_unique dataframe

eco_map_unique["area"] = eco_map_unique.geometry.area

In [ ]:
# create a new dataframe from eco_map_unique where the first column is unique_id and the second column is the area of all the rows in eco_map_unique that have the same unique_id

eco_map_unique_areas = eco_map_unique[["unique_id", "area"]].groupby("unique_id").sum()

In [ ]:
eco_map_unique_areas

In [ ]:
# combine eco_map_unique_areas and overlay_areas into a new dataframe where the first column is unique_id, the second column is area from overlays, and the third column is area from eco_map_unique

combined_areas = pd.concat([overlay_areas, eco_map_unique_areas], axis=1)
combined_areas.columns = ["overlay_area", "eco_map_unique_area"]

combined_areas

In [ ]:
# if overlay_area / eco_map_unique_area > 0.2 then add unique_id to list of ids

native = combined_areas[
    combined_areas["overlay_area"] / combined_areas["eco_map_unique_area"] > 0.2
].index.tolist()

native

In [ ]:
# inter_repro = intersects.to_crs("EPSG:6933")

# inter_repro.crs.axis_info[0].unit_name